In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
import re
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
import warnings
warnings.filterwarnings('ignore')

In [27]:
train = pd.read_csv('data/titanic-2/train.csv')
test = pd.read_csv('data/titanic-2/test.csv')
y_test = pd.read_csv('data/titanic-2/test_pred.csv')['Survived']
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [28]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [29]:
y_test

0      0
1      0
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [12]:
# Create a function to evaluate missing in data, check unique in object (categorical col)
def check_nan(df):

    check = []

    for col in df.columns:

        if df.dtypes[col] == 'object':
            type = 'Object'
            unique_obj = df[col].unique()
        else:
            type = 'Numeric'
            unique_obj = ''

        total_nan = df[col].isnull().sum()
        percent_nan = total_nan/len(df[col])
        check.append([col, type, total_nan, percent_nan, unique_obj])

    check_df = pd.DataFrame(check, columns=['Col', 'Type', 'Total_NaN', 'Percent', 'Unique'])

    return check_df

check = check_nan(train)
check

,Col,Type,Total_NaN,Percent,Unique
0,PassengerId,Numeric,0,0.000000,
1,Survived,Numeric,0,0.000000,
2,Pclass,Numeric,0,0.000000,
3,Name,Object,0,0.000000,"[Braund, Mr. Owen Harris, Cumings, Mrs. John B..."
4,Sex,Object,0,0.000000,"[male, female]"
5,Age,Numeric,177,0.198653,
6,SibSp,Numeric,0,0.000000,
7,Parch,Numeric,0,0.000000,
8,Ticket,Object,0,0.000000,"[A/5 21171, PC 17599, STON/O2. 3101282, 113803..."
9,Fare,Numeric,0,0.000000,


In [13]:
# This is a classification prediction problem, some of best method for prediction is: support vector machine, random forest, boosting, logistic regression, linear discriminant regression, naive bayes, knn 

# Name can take Mr, Miss, Mrs ... out to be a feature, then can take average to fill Age missing (19.8%)

# Convert Sex to Sex_male using dummy

# Ticket numbers: can see some people have same ticket, meaning they grouped together, create ticket_group, can be use for random forest

# Cabin loss more than 70% data, hard to use for traditional, convert nan to U (unknown), take first letter to be new Cabin_class, and filter group people go same cabin as cabin_group like ticket_group

# Embarked convert unknown to U

In [14]:
# Embarked, Cabin convert unknown to U
train['Cabin'].fillna('U', inplace=True)
test['Cabin'].fillna('U', inplace=True)
train['Embarked'].fillna('U', inplace=True)
test['Embarked'].fillna('U', inplace=True)

In [15]:
# Name can take Mr, Miss, Mrs ... out to be a feature, then can take average to fill Age missing (19.8%)
train['Name_p'] = train['Name'].str.extract(r',\s([^.]*)\.')
test['Name_p'] = test['Name'].str.extract(r',\s([^.]*)\.')
unique_name_p = train['Name_p'].value_counts()[(train['Name_p'].value_counts() < 5)]
train[train['Name_p'].map(lambda x: x in unique_name_p)]
# Double check seem okie

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_p
30,31,0,1,"Uruchurtu, Don. Manuel E",male,40.0,0,0,PC 17601,27.7208,U,C,Don
369,370,1,1,"Aubart, Mme. Leontine Pauline",female,24.0,0,0,PC 17477,69.3000,B35,C,Mme
443,444,1,2,"Reynaldo, Ms. Encarnacion",female,28.0,0,0,230434,13.0000,U,S,Ms
449,450,1,1,"Peuchen, Major. Arthur Godfrey",male,52.0,0,0,113786,30.5000,C104,S,Major
536,537,0,1,"Butt, Major. Archibald Willingham",male,45.0,0,0,113050,26.5500,B38,S,Major
556,557,1,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",female,48.0,1,0,11755,39.6000,A16,C,Lady
599,600,1,1,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",male,49.0,1,0,PC 17485,56.9292,A20,C,Sir
641,642,1,1,"Sagesser, Mlle. Emma",female,24.0,0,0,PC 17477,69.3000,B35,C,Mlle
647,648,1,1,"Simonius-Blumer, Col. Oberst Alfons",male,56.0,0,0,13213,35.5000,A26,C,Col
694,695,0,1,"Weir, Col. John",male,60.0,0,0,113800,26.5500,U,S,Col


In [16]:
# Name can take Mr, Miss, Mrs ... out to be a feature, then can take average to fill Age missing (19.8%)
age_concat = pd.concat([train[['Name_p', 'Sex', 'Age']], test[['Name_p', 'Sex', 'Age']]])
age_fit = age_concat[age_concat['Age'].notnull()].groupby(['Name_p', 'Sex']).median().reset_index()

train_null_age = train[train['Age'].isnull()][['Name_p', 'Sex']]
train_null_age = pd.merge(train_null_age, age_fit, on=['Name_p', 'Sex'], how='left')
train.loc[train['Age'].isnull(), 'Age'] = np.array(train_null_age['Age'])

test_null_age = test[test['Age'].isnull()][['Name_p', 'Sex']]
test_null_age = pd.merge(test_null_age, age_fit, on=['Name_p', 'Sex'], how='left')
test.loc[test['Age'].isnull(), 'Age'] = np.array(test_null_age['Age'])

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
 12  Name_p       891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB


In [17]:
test[test['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_p
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,U,S,Mr


In [18]:
test.loc[152, 'Fare'] = train[(train['Pclass']==3) & (train['Age']>50)]['Fare'].mean()

In [19]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_p
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,U,S,Rev
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,22.0,1,2,W./C. 6607,23.4500,U,S,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr


In [20]:
# Ticket numbers: can see some people have same ticket, meaning they grouped together, create ticket_group, can be use for random forest

# Cabin loss more than 70% data, hard to use for traditional, convert nan to U (unknown), take first letter to be new Cabin_class, and filter group people go same cabin as cabin_group like ticket_group

# But decide not use Cabin class because too many loss data, and Pclass also give info about class of passengers

In [21]:
ticket_concat = pd.concat([train['Ticket'],test['Ticket']])
ticket_group = pd.Series(ticket_concat.value_counts()[ticket_concat.value_counts()>1].index)
ticket_group

0              CA. 2343
1               CA 2144
2                  1601
3               3101295
4                347082
             ...       
211             3101278
212    STON/O2. 3101279
213               13050
214              371362
215               36973
Length: 216, dtype: object

In [22]:
def create_group(x, series):
    if x in series.values:
        return True
        # return series[series==x].index[0]
    else:
        return False
        # return -1

train['Ticket_group'] = train['Ticket'].map(lambda x: create_group(x, ticket_group))
test['Ticket_group'] = test['Ticket'].map(lambda x: create_group(x, ticket_group))

In [23]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_p,Ticket_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S,Mr,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,True
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S,Miss,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,True
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S,Mr,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,U,S,Rev,False
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Miss,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,22.0,1,2,W./C. 6607,23.4500,U,S,Miss,True
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Mr,False


In [24]:
cabin_concat = pd.concat([train['Cabin'],test['Cabin']])
cabin_group = pd.Series(cabin_concat.value_counts()[cabin_concat.value_counts()>1].index[1:])
cabin_group

0         C23 C25 C27
1     B57 B59 B63 B66
2                  G6
3                 C78
4                  F4
           ...       
74                E50
75                D21
76                B71
77                D28
78               C123
Length: 79, dtype: object

In [25]:
train['Cabin_group'] = train['Cabin'].map(lambda x: create_group(x, cabin_group))
test['Cabin_group'] = test['Cabin'].map(lambda x: create_group(x, cabin_group))
train['Cabin_class'] = train['Cabin'].map(lambda x: x[:1])
test['Cabin_class'] = test['Cabin'].map(lambda x: x[:1])

In [26]:
list_categoric = ['Sex', 'Embarked', 'Name_p']

train_encode = train.copy()
test_encode = test.copy()
encoder = {}
for col in list_categoric:
    concat_col = pd.concat([train[col],test[col]])
    encoder[col] = OrdinalEncoder().fit(concat_col.values.reshape(-1,1))
    train_encode[col] = encoder[col].transform(train[col].values.reshape(-1,1))
    test_encode[col] = encoder[col].transform(test[col].values.reshape(-1,1))
train_encode

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_p,Ticket_group,Cabin_group,Cabin_class
0,1,0,3,"Braund, Mr. Owen Harris",1.0,22.0,1,0,A/5 21171,7.2500,U,2.0,12.0,False,False,U
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.0,38.0,1,0,PC 17599,71.2833,C85,0.0,13.0,True,True,C
2,3,1,3,"Heikkinen, Miss. Laina",0.0,26.0,0,0,STON/O2. 3101282,7.9250,U,2.0,9.0,False,False,U
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.0,35.0,1,0,113803,53.1000,C123,2.0,13.0,True,True,C
4,5,0,3,"Allen, Mr. William Henry",1.0,35.0,0,0,373450,8.0500,U,2.0,12.0,False,False,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1.0,27.0,0,0,211536,13.0000,U,2.0,15.0,False,False,U
887,888,1,1,"Graham, Miss. Margaret Edith",0.0,19.0,0,0,112053,30.0000,B42,2.0,9.0,False,False,B
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0.0,22.0,1,2,W./C. 6607,23.4500,U,2.0,9.0,True,False,U
889,890,1,1,"Behr, Mr. Karl Howell",1.0,26.0,0,0,111369,30.0000,C148,0.0,12.0,False,False,C


In [30]:
features_random_forest = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Name_p', 'Ticket_group']
X_train_rdn_forest = train_encode[features_random_forest]
y_train_rdn_forest = train_encode['Survived']
X_test_rdn_forest = test_encode[features_random_forest]

In [31]:
# Create Bagging or Random Forests. Bagging just a special case of random forest with number of features use = total p. Optimal Random Forests features use m = squared root of p
random_forest = RandomForestClassifier(max_features='sqrt', random_state=1, n_estimators=500)
random_forest.fit(X_train_rdn_forest, y_train_rdn_forest)

# Return training accuracy
print('Accuracy training score: {:.4f}'.format(random_forest.score(X_train_rdn_forest, y_train_rdn_forest)))

Accuracy training score: 0.9798


In [22]:
# grid_param = {
#  'n_estimators': [200],
#  'criterion':['gini', 'entropy'],
#  'min_samples_split': [2, 10, 20],
#  'min_samples_leaf': [1, 5]
# }
# gd_sr = GridSearchCV(estimator=RandomForestClassifier(),
#  param_grid=grid_param,
#  scoring='accuracy',
#  cv=5)
# gd_sr.fit(X_train_rdn_forest, y_train_rdn_forest)
# best_parameters = gd_sr.best_params_
# print(best_parameters)

In [32]:
# Using Cross-validation for random forest:
random_forest = RandomForestClassifier(min_samples_leaf=1, min_samples_split=10, random_state=1, n_estimators=200, criterion='gini')
score = cross_val_score(random_forest, X_train_rdn_forest, y_train_rdn_forest, cv=10, scoring='accuracy')
score.mean()

0.8339450686641697

In [33]:
# Using Cross-validation for random forest:
boosting = GradientBoostingClassifier(n_estimators=1000, max_depth=5, learning_rate=0.05, random_state=1)
score = cross_val_score(boosting, X_train_rdn_forest, y_train_rdn_forest, cv=10, scoring='accuracy')
score.mean()

0.8092634207240949

In [34]:
# Create Boosting with d = 1, lamda = 0.01
boosting = GradientBoostingClassifier(n_estimators=1000, max_depth=5, learning_rate=0.05, random_state=1)
boosting.fit(X_train_rdn_forest, y_train_rdn_forest)

# Return training accuracy
print('Accuracy training score: {:.4f}'.format(boosting.score(X_test_rdn_forest, y_test)))

Accuracy training score: 0.8254
